In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-darkgrid')

print('Phase 7 Libraries loaded')

Phase 7 Libraries loaded


## Load Data with Lagged Features

In [2]:
file_path = 'd:/S2/prediksi - hujan/merged_all_data_complete.csv'
df = pd.read_csv(file_path)
df['valid_time'] = pd.to_datetime(df['valid_time'])
df = df.sort_values('valid_time').reset_index(drop=True)

# Create daily aggregation
daily_data = df.groupby(df['valid_time'].dt.date)[['tp', 'ro', 't2m', 'u10', 'v10', 'swvl1', 'wind_speed']].mean()
daily_data.index = pd.to_datetime(daily_data.index)

# Add lagged features (from Phase 4)
for col in ['tp', 'ro', 't2m', 'u10', 'v10', 'swvl1', 'wind_speed']:
    for lag in [1, 2, 3, 6, 7, 14]:
        daily_data[f'{col}_lag{lag}'] = daily_data[col].shift(lag)

print(f'Daily data with lags: {daily_data.shape}')
print(f'Columns: {daily_data.columns.tolist()}')
print(f'First row with complete lags (row 15):')
print(daily_data.iloc[14])

Daily data with lags: (1827, 49)
Columns: ['tp', 'ro', 't2m', 'u10', 'v10', 'swvl1', 'wind_speed', 'tp_lag1', 'tp_lag2', 'tp_lag3', 'tp_lag6', 'tp_lag7', 'tp_lag14', 'ro_lag1', 'ro_lag2', 'ro_lag3', 'ro_lag6', 'ro_lag7', 'ro_lag14', 't2m_lag1', 't2m_lag2', 't2m_lag3', 't2m_lag6', 't2m_lag7', 't2m_lag14', 'u10_lag1', 'u10_lag2', 'u10_lag3', 'u10_lag6', 'u10_lag7', 'u10_lag14', 'v10_lag1', 'v10_lag2', 'v10_lag3', 'v10_lag6', 'v10_lag7', 'v10_lag14', 'swvl1_lag1', 'swvl1_lag2', 'swvl1_lag3', 'swvl1_lag6', 'swvl1_lag7', 'swvl1_lag14', 'wind_speed_lag1', 'wind_speed_lag2', 'wind_speed_lag3', 'wind_speed_lag6', 'wind_speed_lag7', 'wind_speed_lag14']
First row with complete lags (row 15):
tp                    0.000025
ro                    0.000013
t2m                 299.533559
u10                  -3.383728
v10                  -1.147465
swvl1                 0.088697
wind_speed            3.959040
tp_lag1               0.000022
tp_lag2               0.000037
tp_lag3               0.000026

## Feature 1: Cumulative Rainfall Windows

In [3]:
# Cumulative rainfall over different windows
daily_data['tp_cumsum_1d'] = daily_data['tp'].rolling(window=1, min_periods=1).sum()
daily_data['tp_cumsum_3d'] = daily_data['tp'].rolling(window=3, min_periods=1).sum()
daily_data['tp_cumsum_7d'] = daily_data['tp'].rolling(window=7, min_periods=1).sum()
daily_data['tp_cumsum_14d'] = daily_data['tp'].rolling(window=14, min_periods=1).sum()
daily_data['tp_cumsum_30d'] = daily_data['tp'].rolling(window=30, min_periods=1).sum()

print('Cumulative rainfall features created:')
print(daily_data[['tp', 'tp_cumsum_1d', 'tp_cumsum_3d', 'tp_cumsum_7d', 'tp_cumsum_14d', 'tp_cumsum_30d']].head(40))

Cumulative rainfall features created:
                  tp  tp_cumsum_1d  tp_cumsum_3d  tp_cumsum_7d  tp_cumsum_14d  \
valid_time                                                                      
2020-01-01  0.000036      0.000036      0.000036      0.000036       0.000036   
2020-01-02  0.000018      0.000018      0.000054      0.000054       0.000054   
2020-01-03  0.000033      0.000033      0.000087      0.000087       0.000087   
2020-01-04  0.000010      0.000010      0.000062      0.000098       0.000098   
2020-01-05  0.000016      0.000016      0.000059      0.000113       0.000113   
2020-01-06  0.000012      0.000012      0.000038      0.000126       0.000126   
2020-01-07  0.000010      0.000010      0.000038      0.000135       0.000135   
2020-01-08  0.000013      0.000013      0.000035      0.000113       0.000149   
2020-01-09  0.000080      0.000080      0.000103      0.000174       0.000228   
2020-01-10  0.000028      0.000028      0.000120      0.000169       0.

## Feature 2: Rainfall Intensity & Change Rate

In [4]:
# Rainfall intensity (current - previous day)
daily_data['tp_change_1d'] = daily_data['tp'].diff(1)
daily_data['tp_change_3d'] = daily_data['tp'].diff(3)

# Rainfall rate of change (intensity trend)
daily_data['tp_intensity_3d'] = daily_data['tp'].rolling(window=3, min_periods=1).mean()
daily_data['tp_intensity_7d'] = daily_data['tp'].rolling(window=7, min_periods=1).mean()

# Runoff change
daily_data['ro_change_1d'] = daily_data['ro'].diff(1)
daily_data['ro_cumsum_3d'] = daily_data['ro'].rolling(window=3, min_periods=1).sum()

print('Rainfall intensity features created:')
print(daily_data[['tp', 'tp_change_1d', 'tp_change_3d', 'tp_intensity_3d', 'tp_intensity_7d']].head(20))

Rainfall intensity features created:
                  tp  tp_change_1d  tp_change_3d  tp_intensity_3d  \
valid_time                                                          
2020-01-01  0.000036           NaN           NaN         0.000036   
2020-01-02  0.000018     -0.000018           NaN         0.000027   
2020-01-03  0.000033      0.000015           NaN         0.000029   
2020-01-04  0.000010     -0.000023 -2.558231e-05         0.000021   
2020-01-05  0.000016      0.000005 -2.551079e-06         0.000020   
2020-01-06  0.000012     -0.000003 -2.076626e-05         0.000013   
2020-01-07  0.000010     -0.000003 -5.984307e-07         0.000013   
2020-01-08  0.000013      0.000004 -2.460480e-06         0.000012   
2020-01-09  0.000080      0.000066  6.728411e-05         0.000034   
2020-01-10  0.000028     -0.000052  1.778364e-05         0.000040   
2020-01-11  0.000042      0.000014  2.847910e-05         0.000050   
2020-01-12  0.000026     -0.000016 -5.352974e-05         0.000032 

## Feature 3: Soil Water Dynamics

In [5]:
# Soil water change
daily_data['swvl1_change_1d'] = daily_data['swvl1'].diff(1)
daily_data['swvl1_change_3d'] = daily_data['swvl1'].diff(3)

# Soil saturation indicator (relative to rolling max)
daily_data['swvl1_saturation_7d'] = daily_data['swvl1'] / daily_data['swvl1'].rolling(window=7, min_periods=1).max()

# Combined soil-rainfall indicator
daily_data['soil_rainfall_interaction'] = daily_data['tp'] * daily_data['swvl1_saturation_7d']

print('Soil water dynamics features created:')
print(daily_data[['swvl1', 'swvl1_change_1d', 'swvl1_change_3d', 'swvl1_saturation_7d', 'soil_rainfall_interaction']].head(20))

Soil water dynamics features created:
               swvl1  swvl1_change_1d  swvl1_change_3d  swvl1_saturation_7d  \
valid_time                                                                    
2020-01-01  0.121913              NaN              NaN             1.000000   
2020-01-02  0.117451        -0.004462              NaN             0.963402   
2020-01-03  0.114236        -0.003215              NaN             0.937035   
2020-01-04  0.111112        -0.003124        -0.010801             0.911407   
2020-01-05  0.108178        -0.002934        -0.009273             0.887343   
2020-01-06  0.105212        -0.002967        -0.009025             0.863010   
2020-01-07  0.101502        -0.003710        -0.009610             0.832578   
2020-01-08  0.098283        -0.003219        -0.009896             0.836799   
2020-01-09  0.101329         0.003046        -0.003883             0.887010   
2020-01-10  0.102136         0.000807         0.000634             0.919213   
2020-01-11  0.

## Feature 4: Temperature Features

In [6]:
# Temperature anomaly (deviation from rolling mean)
daily_data['t2m_anomaly_7d'] = daily_data['t2m'] - daily_data['t2m'].rolling(window=7, min_periods=1).mean()
daily_data['t2m_anomaly_30d'] = daily_data['t2m'] - daily_data['t2m'].rolling(window=30, min_periods=1).mean()

# Temperature change
daily_data['t2m_change_1d'] = daily_data['t2m'].diff(1)
daily_data['t2m_change_3d'] = daily_data['t2m'].diff(3)

print('Temperature features created:')
print(daily_data[['t2m', 't2m_anomaly_7d', 't2m_anomaly_30d', 't2m_change_1d', 't2m_change_3d']].head(20))

Temperature features created:
                   t2m  t2m_anomaly_7d  t2m_anomaly_30d  t2m_change_1d  \
valid_time                                                               
2020-01-01  299.637534        0.000000         0.000000            NaN   
2020-01-02  299.589028       -0.024253        -0.024253      -0.048506   
2020-01-03  299.655711        0.028287         0.028287       0.066684   
2020-01-04  299.594656       -0.024577        -0.024577      -0.061056   
2020-01-05  299.489174       -0.104047        -0.104047      -0.105482   
2020-01-06  299.364958       -0.190219        -0.190219      -0.124216   
2020-01-07  299.428738       -0.108376        -0.108376       0.063780   
2020-01-08  299.654798        0.115218         0.102973       0.226060   
2020-01-09  299.317831       -0.183007        -0.207994      -0.336967   
2020-01-10  299.531238        0.048182         0.004871       0.213407   
2020-01-11  299.081975       -0.327841        -0.403993      -0.449263   
2020-01-

## Feature 5: Wind Dynamics

In [7]:
# Wind speed change
daily_data['wind_speed_change_1d'] = daily_data['wind_speed'].diff(1)
daily_data['wind_speed_change_3d'] = daily_data['wind_speed'].diff(3)

# Wind anomaly
daily_data['wind_speed_anomaly_7d'] = daily_data['wind_speed'] - daily_data['wind_speed'].rolling(window=7, min_periods=1).mean()

# Wind acceleration (change in change)
daily_data['wind_accel_1d'] = daily_data['wind_speed_change_1d'].diff(1)

print('Wind dynamics features created:')
print(daily_data[['wind_speed', 'wind_speed_change_1d', 'wind_speed_change_3d', 'wind_speed_anomaly_7d', 'wind_accel_1d']].head(20))

Wind dynamics features created:
            wind_speed  wind_speed_change_1d  wind_speed_change_3d  \
valid_time                                                           
2020-01-01    5.424700                   NaN                   NaN   
2020-01-02    5.139974             -0.284726                   NaN   
2020-01-03    4.405949             -0.734025                   NaN   
2020-01-04    4.725085              0.319136             -0.699614   
2020-01-05    4.277579             -0.447506             -0.862395   
2020-01-06    4.545137              0.267558              0.139188   
2020-01-07    4.784129              0.238992              0.059044   
2020-01-08    4.575088             -0.209041              0.297509   
2020-01-09    3.496361             -1.078726             -1.048775   
2020-01-10    2.159589             -1.336773             -2.624540   
2020-01-11    2.277213              0.117624             -2.297875   
2020-01-12    3.010352              0.733139             -

## Feature 6: Temporal Cyclical Encoding

In [8]:
# Extract temporal information from index
daily_data['day_of_year'] = daily_data.index.dayofyear
daily_data['month'] = daily_data.index.month
daily_data['day_of_week'] = daily_data.index.dayofweek
daily_data['week_of_year'] = daily_data.index.isocalendar().week

# Cyclical encoding using sin/cos (preserves circularity)
daily_data['month_sin'] = np.sin(2 * np.pi * daily_data['month'] / 12)
daily_data['month_cos'] = np.cos(2 * np.pi * daily_data['month'] / 12)

daily_data['day_of_year_sin'] = np.sin(2 * np.pi * daily_data['day_of_year'] / 365)
daily_data['day_of_year_cos'] = np.cos(2 * np.pi * daily_data['day_of_year'] / 365)

daily_data['day_of_week_sin'] = np.sin(2 * np.pi * daily_data['day_of_week'] / 7)
daily_data['day_of_week_cos'] = np.cos(2 * np.pi * daily_data['day_of_week'] / 7)

print('Temporal cyclical encoding features created:')
print(daily_data[['month', 'month_sin', 'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'day_of_week_sin', 'day_of_week_cos']].head(20))

Temporal cyclical encoding features created:
            month  month_sin  month_cos  day_of_year_sin  day_of_year_cos  \
valid_time                                                                  
2020-01-01      1        0.5   0.866025         0.017213         0.999852   
2020-01-02      1        0.5   0.866025         0.034422         0.999407   
2020-01-03      1        0.5   0.866025         0.051620         0.998667   
2020-01-04      1        0.5   0.866025         0.068802         0.997630   
2020-01-05      1        0.5   0.866025         0.085965         0.996298   
2020-01-06      1        0.5   0.866025         0.103102         0.994671   
2020-01-07      1        0.5   0.866025         0.120208         0.992749   
2020-01-08      1        0.5   0.866025         0.137279         0.990532   
2020-01-09      1        0.5   0.866025         0.154309         0.988023   
2020-01-10      1        0.5   0.866025         0.171293         0.985220   
2020-01-11      1        0.5   

## Feature Summary & Data Quality Check

In [9]:
print('='*70)
print('FEATURE ENGINEERING COMPLETE')
print('='*70)

print(f'\nTotal features created: {daily_data.shape[1]}')
print(f'Rows: {daily_data.shape[0]}')

# Count NaN values
nan_counts = daily_data.isna().sum()
print(f'\nNaN values per feature (top 10):')
print(nan_counts.nlargest(10))

print(f'\nFeature categories:')
print(f'  - Original variables: 7')
print(f'  - Lagged features: 12 (7 variables x 6 lags)')
print(f'  - Cumulative rainfall: 5')
print(f'  - Intensity & change: 6')
print(f'  - Soil dynamics: 4')
print(f'  - Temperature: 4')
print(f'  - Wind dynamics: 4')
print(f'  - Temporal cyclical: 9')
print(f'  - Total new features: {daily_data.shape[1] - 7}')

# Sample of engineered features
print(f'\nSample engineered features (row 30):')
print(daily_data.iloc[29][['tp', 'tp_cumsum_7d', 'tp_intensity_7d', 'swvl1_saturation_7d', 'soil_rainfall_interaction', 'month_sin', 'wind_accel_1d']])

FEATURE ENGINEERING COMPLETE

Total features created: 82
Rows: 1827

NaN values per feature (top 10):
tp_lag14            14
ro_lag14            14
t2m_lag14           14
u10_lag14           14
v10_lag14           14
swvl1_lag14         14
wind_speed_lag14    14
tp_lag7              7
ro_lag7              7
t2m_lag7             7
dtype: int64

Feature categories:
  - Original variables: 7
  - Lagged features: 12 (7 variables x 6 lags)
  - Cumulative rainfall: 5
  - Intensity & change: 6
  - Soil dynamics: 4
  - Temperature: 4
  - Wind dynamics: 4
  - Temporal cyclical: 9
  - Total new features: 75

Sample engineered features (row 30):
tp                           0.000047
tp_cumsum_7d                  0.00042
tp_intensity_7d               0.00006
swvl1_saturation_7d               1.0
soil_rainfall_interaction    0.000047
month_sin                         0.5
wind_accel_1d                1.093149
Name: 2020-01-30 00:00:00, dtype: Float64

NaN values per feature (top 10):
tp_lag14       

## Save Engineered Dataset

In [10]:
# Save engineered features dataset
output_path = 'd:/S2/prediksi - hujan/engineered_features_dataset.csv'
daily_data.to_csv(output_path)

print(f'Engineered features dataset saved to:')
print(f'{output_path}')
print(f'\nDataset info:')
print(f'  - Rows: {daily_data.shape[0]}')
print(f'  - Columns: {daily_data.shape[1]}')
print(f'  - File size: {daily_data.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB')
print(f'  - Date range: {daily_data.index.min().date()} to {daily_data.index.max().date()}')

Engineered features dataset saved to:
d:/S2/prediksi - hujan/engineered_features_dataset.csv

Dataset info:
  - Rows: 1827
  - Columns: 82
  - File size: 1.13 MB
  - Date range: 2020-01-01 to 2024-12-31
